In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta


def get_multi_index_prices(end_date: str) -> pd.DataFrame:
    """
    여러 지표의 1년치 Close 가격을 하나의 DataFrame으로 반환
    (Date, VIX, S&P500, NASDAQ, DXY, DOWJONES, US10Y)
    """
    tickers = {
        "DXY": "DX-Y.NYB",
        "NASDAQ": "^IXIC",
        "S&P500": "^GSPC",
        "DOWJONES": "^DJI",
        "VIX": "^VIX",
        "US10Y": "^TNX"
    }

    end = datetime.strptime(end_date, "%Y-%m-%d")
    start = end - timedelta(days=365)

    df = yf.download(
        list(tickers.values()),
        start=start.strftime("%Y-%m-%d"),
        end=end.strftime("%Y-%m-%d"),
        progress=False
    )["Close"]

    # 멀티인덱스 컬럼이면 풀기
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)

    # 컬럼명 매핑
    rename_map = {v: k for k, v in tickers.items()}
    df = df.rename(columns=rename_map)

    # 원하는 순서
    ordered_cols = ["VIX", "S&P500", "NASDAQ", "DXY", "DOWJONES", "US10Y"]
    df = df[ordered_cols]

    # 컬럼 인덱스 이름 제거 → "Ticker" 안 뜨게 함
    df.columns.name = None

    return df

# 실행 예시
# if __name__ == "__main__":
#     df_indices = get_multi_index_prices("2025-01-02")
#     print(df_indices.tail())


c
    

In [ ]:
get_quarterly_report(symbol: str, date: str)
get_price_history(ticker: str, end_date: str)
get_multi_index_prices(end_date: str)

In [ ]:
get_quarterly_report('AAPL', '2025-6-20')

In [ ]:
df = get_quarterly_report('RZLV', '2025-10-03')

In [ ]:
get_price_history('RZLV', '2025-10-03')

In [ ]:
get_multi_index_prices('2025-10-03')


In [ ]:
def get_quarterly_report(symbol: str, date: str, period: int) -> pd.DataFrame:
    tk = yf.Ticker(symbol)
    target_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = target_date - timedelta(days=365*period)

    # 재무 데이터
    income = tk.quarterly_financials.T
    balance = tk.quarterly_balance_sheet.T
    cashflow = tk.quarterly_cashflow.T

    # 기간 필터링
    valid_periods = [p for p in income.index if p + timedelta(days=45) <= target_date and p >= start_date]
    print(valid_periods)
    if not valid_periods:
        return pd.DataFrame()

    df_list = []
    for p in valid_periods:
        row = {
            "symbol": symbol,
            "period": p,
            "year": p.year,
            "net_income": income.loc[p].get("Net Income"),
            "revenue": income.loc[p].get("Total Revenue"),
            "operating_income": income.loc[p].get("Operating Income"),
            "gross_profit": income.loc[p].get("Gross Profit"),
            "total_assets": balance.loc[p].get("Total Assets"),
            "total_liabilities": balance.loc[p].get("Total Liabilities"),
            "operating_cf": cashflow.loc[p].get("Total Cash From Operating Activities"),
            "capex": cashflow.loc[p].get("Capital Expenditures"),
        }
        df_list.append(row)

    df = pd.DataFrame(df_list).sort_values("period")

    # 파생 변수 예시
    df["profit_margin"] = df["net_income"] / df["revenue"]
    df["debt_to_equity"] = df["total_liabilities"] / (df["total_assets"] - df["total_liabilities"])
    df["free_cf"] = (df["operating_cf"].fillna(0) + df["capex"].fillna(0))

    # 최근 분기 값, 최근 4분기 평균, 최근 12분기 평균
    summary = {
        "net_income_latest": df["net_income"].iloc[-1],
        "net_income_avg4": df["net_income"].tail(4).mean(),
        "net_income_avg12": df["net_income"].tail(12).mean(),
        "revenue_latest": df["revenue"].iloc[-1],
        "revenue_avg4": df["revenue"].tail(4).mean(),
        "revenue_avg12": df["revenue"].tail(12).mean(),
    }

    return df


In [ ]:
get_quarterly_report('TSLA', '2025-10-03', 3)

In [ ]:
tk = yf.Ticker('RZLV')

In [ ]:
tk.balance_sheet

In [ ]:
tk.recommendations

In [ ]:
tk.recommendations_summary

In [ ]:
tk.analyst_price_target

In [ ]:
tk.upgrades_downgrades  

In [ ]:
a = tk.history(period='2d', interval='15m')
a

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.plot(a.index, a)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 예시 데이터
data = {
    "Datetime": [
        "2025-10-01 09:30:00", "2025-10-01 09:45:00", "2025-10-01 10:00:00"
    ],
    "Open": [6.03, 5.78, 5.87],
    "High": [6.23, 5.97, 5.94],
    "Low": [5.61, 5.71, 5.73],
    "Close": [5.77, 5.87, 5.78],
    "Volume": [19404509, 2928167, 1622564],
}
df = pd.DataFrame(a)
df["Datetime"] = pd.to_datetime(df["Datetime"])
df.set_index("Datetime", inplace=True)

# 캔들차트 + 거래량
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6), sharex=True,
                               gridspec_kw={'height_ratios':[3,1]})

for dt, row in df.iterrows():
    color = 'red' if row['Close'] >= row['Open'] else 'blue'
    # 고가~저가 (wick)
    ax1.plot([dt, dt], [row['Low'], row['High']], color=color)
    # 시가~종가 (body)
    ax1.add_patch(plt.Rectangle((dt, min(row['Open'], row['Close'])),
                                pd.Timedelta(minutes=15),  # 캔들 너비
                                abs(row['Close']-row['Open']),
                                color=color))
ax1.set_ylabel("Price")
ax1.set_title("Candlestick with Volume")

# 거래량
ax2.bar(df.index, df["Volume"], width=600, color="gray")
ax2.set_ylabel("Volume")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import yfinance as yf
import mplfinance as mpf

# 애플 주가 데이터 다운로드
df = yf.download("AAPL", period="3mo", interval="1d")
ㅠ
# 캔들차트 + 거래량
mpf.plot(df, type="candle", volume=True, style="charles")


In [ ]:
df

In [ ]:
import yfinance as yf
import mplfinance as mpf

# 데이터 다운로드 (auto_adjust=False로 원래처럼 조정 안 한 가격 받기)
df = yf.download("RZLV", period="3mo", interval="1d", auto_adjust=False)

# 불필요한 컬럼 제거
df = df.drop(columns=["Dividends","Stock Splits"], errors="ignore")

# NaN 제거
df = df.dropna()

# 인덱스 타임존 제거
df.index = df.index.tz_localize(None)

# 혹시 MultiIndex로 내려온 경우 대비 → 단일 레벨로 강제 변환
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# 컬럼명 표준화 (대문자 확인)
df = df.rename(columns=str.title)  # 'open' → 'Open'

print(df.dtypes)   # 모든 컬럼 float64 확인
print(df.head())

# 캔들차트 + 거래량
mpf.plot(df, type="candle", volume=True, style="charles", figsize=(10,6))


In [ ]:
df

In [ ]:
import yfinance as yf
import plotly.graph_objects as go

# 데이터 다운로드
df = yf.download("TSLA", period="3mo", interval="1d", auto_adjust=False)

# 혹시 MultiIndex로 내려온 경우 대비 → 단일 레벨로 강제 변환
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
    
# 캔들차트 (y축 = Price)
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name="Candlestick",
    yaxis="y1"
))

# 거래량 (y축 = Volume, 오른쪽)
fig.add_trace(go.Bar(
    x=df.index,
    y=df['Volume'],
    name="Volume",
    marker_color="lightgray",
    yaxis="y2"
))

# 레이아웃
fig.update_layout(
    title="RZLV Candlestick with Volume",
    xaxis=dict(rangeslider=dict(visible=False)),
    yaxis=dict(title="Price", side="left"),
    yaxis2=dict(title="Volume", side="right", overlaying="y", showgrid=False),
    template="plotly_white",
    hovermode="x unified",
    height=600
)

fig.show()

In [ ]:
# streamlit_app.py
import yfinance as yf
import plotly.graph_objects as go
import streamlit as st

ticker = st.text_input("Ticker", "AAPL")
df = yf.download(ticker, period="6mo", interval="1d", auto_adjust=False)

fig = go.Figure(data=[go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']
)])
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume", marker_color="lightgray"))

st.plotly_chart(fig, use_container_width=True)


In [ ]:
df

In [ ]:
tsla = yf.Ticker("TSLA")